<a href="https://colab.research.google.com/github/Utomi-Tom/Movie-Binary-classification/blob/main/IMBD_movie_classification_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The IMDB dataset
You’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the
Internet Movie Database. They’re split into 25,000 reviews for training and 25,000
reviews for testing, each set consisting of 50% negative and 50% positive reviews.


Why use separate training and test sets? Because you should never test a machinelearning
model on the same data that you used to train it! Just because a model performs
well on its training data doesn’t mean it will perform well on data it has never
seen; and what you care about is your model’s performance on new data (because you
already know the labels of your training data—obviously you don’t need your model
to predict those). For instance, it’s possible that your model could end up merely memorizing
a mapping between your training samples and their targets, which would be
useless for the task of predicting targets for data the model has never seen before.
We’ll go over this point in much more detail in the next chapter.


Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has
already been preprocessed: the reviews (sequences of words) have been turned into
sequences of integers, where each integer stands for a specific word in a dictionary.
The following code will load the dataset (when you run it the first time, about
80 MB of data will be downloaded to your machine).

In [2]:
import tensorflow as tf
from tensorflow import keras

In [7]:
# Import dataset 
(x_tr, y_tr), (x_tt, y_tt) = keras.datasets.imdb.load_data(num_words=10000)

The argument num_words=10000 means you’ll only keep the top 10,000 most frequently
occurring words in the training data. Rare words will be discarded. This allows
you to work with vector data of manageable size.

In [14]:
word_index = keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


here’s how you can quickly decode one of these reviews back to English
words:

word_index is a dictionary mapping
words to an integer index.

In [15]:
r_word_ind = dict([(value, key) for (key, value) in word_index.items()])

Reverses it, mapping
integer indices to words

In [19]:
decoded_review = "".join([r_word_ind.get(i - 3, "?") for i in x_tr[3]])

In [10]:
(x_tr[0])

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

The variables **x_tr and x_tt** are lists of reviews; each review is a list of
word indices (encoding a sequence of words).**y_tr and y_tt** are
lists of 0s and 1s, where 0 stands for negative and 1 stands for positive:

In [11]:
y_tr

array([1, 0, 0, ..., 0, 1, 0])

In [20]:
decoded_review


"?the??atstorytellingthetraditionalsortmanyyearsaftertheeventicanstillseeinmy?eyeanelderlyladymyfriend'smotherretellingthebattleof?shemakesthecharacterscomealiveherpassionisthatofaneyewitnessonetotheeventsonthe?heathamileorsofromwhereshelivesbrbrofcourseithappenedmanyyearsbeforeshewasbornbutyouwouldn'tguessfromthewayshetellsitthesamestoryistoldinbarsthelengthand?ofscotlandasidiscusseditwithafriendonenightin?alocalcutintogivehisversionthediscussioncontinuedtoclosingtimebrbrstoriespasseddownlikethisbecomepartofourbeingwhodoesn'trememberthestoriesourparentstolduswhenwewerechildrentheybecomeourinvisibleworldandaswegrowoldertheymaybestillserveasinspirationorasanemotional?factandfictionblendwith?rolemodelswarningstories?magicandmysterybrbrmynameis?likemygrandfatherandhisgrandfatherbeforehimourprotagonistintroduceshimselftousandalsointroducesthestorythatstretchesbackthroughgenerationsitproducesstorieswithinstoriesstoriesthatevokethe?wonderofscotlanditsruggedmountains?in?thestuffoflegendyet?is

# Preparing the data